# DAIN Colab alpha

Credits to original Colab file: 
https://github.com/baowenbo/DAIN/issues/44

My fork:
https://github.com/styler00dollar/DAIN

Enhancement by Styler00Dollar aka "sudo rm -rf / --no-preserve-root#8353" on discord and Alpha. Please do not run this command in your linux terminal. It's rather meant as a joke.

A simple guide:
- Copy the .ipynb-file to your drive.
- Create a folder inside of Google Drive named "DAIN"
- Don't forget to configure the filename in Colab. You will notice that down below.

Stuff that should be improved:
- Gif (RGBA24 or alpha in general) is currently not supported.
- Adding menu to select speed.

In [0]:
################# Configuration cell! ############################
# Use the values in here to configure what you'd like DAIN to do.

# Input file: Path (relative to the root of your Google Drive) to the input file.
# For instance, if you save your "example.mkv" file in your Google Drive, inside a "videos" folder, the path would be:
# videos/example.mkv
INPUT_FILEPATH = "DAIN/input.mp4"

# Target FPS = how many frames per second should the result have. This will determine how many intermediate images are
# interpolated.
TARGET_FPS = 60

# Output file path: path (relative to the root of your Google Drive) for the output file.
# Extension should always be MKV.
OUTPUT_FILE_PATH = "DAIN/output.mkv"

In [0]:
# Connect Google Drive

from google.colab import drive
drive.mount('/content/gdrive')
model_dir = '/content/gdrive/My Drive/DAIN/'
print('Google Drive connected.')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Google Drive connected.


In [0]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# P100: 16GB (Can handle 720p. 1080p does not work and you will get an out-of-memory error.)
# P4: 8GB (Not tested.)
# K80: 8GB (Not tested.)

!nvidia-smi --query-gpu=gpu_name --format=csv,noheader

Tesla P100-PCIE-16GB


In [0]:
#@title Inititialization (Run this once at startup. Approximately 15-20 Minutes.)

from IPython.display import clear_output
!rm -rf /content/DAIN
!git clone https://github.com/styler00dollar/Colab-DAIN

# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/DAIN/my_package/
!./build.sh
#clear_output()
print("Building #1 done.")

# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
#clear_output()
print("Building #2 done.")

# Downloading pre-trained models
%cd /content/DAIN
!mkdir model_weights MiddleBurySet
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

%cd /content/DAIN/MiddleBurySet
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
!unzip other-color-allframes.zip

!wget http://vision.middlebury.edu/flow/data/comp/zip/other-gt-interp.zip
!unzip other-gt-interp.zip
#clear_output()

!CUDA_VISIBLE_DEVICES=0

# Fix scipy
# DO NOT RESTART YOUR COLAB!! Yes, there is red text and a warning, but it works.

!pip install --force-reinstall scipy==1.0.0

Building #1 done.
/content/DAIN/PWCNet/correlation_package_pytorch1_0


MessageError: ignored

In [61]:
# Detecting FPS of input file.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

# ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf /content/DAIN/input_frames
%shell mkdir -p /content/DAIN/input_frames

# TODO: Work with this to remove the alpha channel
#%shell ffmpeg -i '/content/DAIN/{filename}' -filter_complex 'split [rgb_in][alpha_in]; [rgb_in][out];' '/content/DAIN/input_frames/%05d.png'

%shell ffmpeg -i '/content/DAIN/{filename}' '/content/DAIN/input_frames/%05d.png'

png_generated_count_command_result = %shell ls /content/DAIN/input_frames | wc -l
clear_output()

pngs_generated_count = int(png_generated_count_command_result.output.strip())
print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

Input FPS: 10.0
24 frame PNGs generated.


In [64]:
# Interpolation
%shell rm -rf /content/DAIN/output_frames
%shell mkdir /content/DAIN/output_frames
%cd /content/DAIN
#!python colab_MiddleBury_slowmotion.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --save_which=0
!python colab_MiddleBury_slowmotion.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS}

/content/DAIN
revise the unique id to a random numer 80195
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/80195-Mon-Mar-09-17:24/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, log='./model_weights/80195-Mon-Mar-09-17:24/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/80195-Mon-Mar-09-17:24', save_which=1, seed=1, time_step=0.16666666666666666, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 5 frames
The testing model weight is: ./model_weights/best.pth
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to 

In [65]:
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -i '/content/DAIN/output_frames/%05d.png' output.mkv

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
# Copy result to drive
!cp /content/DAIN/output.mkv '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'